In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](CRISP_DM.png)

# Groupby apply on large (relational) data set

### Assumption -dataframe with sorted data fot all written functions


In [2]:
%cd ads_covid-19/data/

D:\CVT\EDS_SS2022_Salim\ads_covid-19\data


In [3]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,Kosovo,0.0
2,2020-01-22,no,Kuwait,0.0
3,2020-01-22,no,Kyrgyzstan,0.0
4,2020-01-22,no,Laos,0.0


# Test data


In [4]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [5]:
test_data.head()

,date,state,country,confirmed
16854,2020-03-21,no,Germany,16662.0
16918,2020-03-21,no,US,26025.0
17139,2020-03-22,no,Germany,18610.0
17189,2020-03-22,no,US,34944.0
17424,2020-03-23,no,Germany,22672.0


In [6]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2022-05-20,no,26040460.0
US,2022-05-20,no,83237592.0


In [7]:
# build features for small dataset
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [8]:
test_data.groupby(['state','country']).agg(np.max)

date   confirmed
state country                       
no    Germany 2022-05-20  26040460.0
      US      2022-05-20  83237592.0

In [9]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Germany  16854            NaN
                17139            NaN
                17424       6.427510
                17709       5.190573
                17995       6.129400
                             ...    
       US       240914    936.817919
                241213    641.020854
                241491    515.940794
                241769    521.560337
                242066    580.569785
Name: confirmed, Length: 1582, dtype: float64

In [11]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [14]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,level_0,index,date,state,country,confirmed
0,0,0,2020-01-22,Alberta,Canada,0.0
1,1,1,2020-01-22,no,Kosovo,0.0
2,2,2,2020-01-22,no,Kuwait,0.0
3,3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,4,2020-01-22,no,Laos,0.0


In [15]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,level_0,index,date,state,country,confirmed,confirmed_DR
0,0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,1,2020-01-22,no,Kosovo,0.0,NaN
2,2,2,2020-01-22,no,Kuwait,0.0,NaN
3,3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN
4,4,4,2020-01-22,no,Laos,0.0,NaN


In [16]:
# for Germany if needed
#pd_result_larg[pd_result_larg['country']=='Germany']

## Filtering the data with groupby apply

In [17]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [18]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0
2,2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0
3,3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0
4,4,4,2020-01-22,no,Laos,0.0,NaN,0.0


## Filtered doubling rate

In [20]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
242245,no,Zimbabwe,240966,1483.338092
242246,no,Zimbabwe,241179,1330.105899
242247,no,Zimbabwe,241486,1103.566652
242248,no,Zimbabwe,241728,988.033465
242249,no,Zimbabwe,242249,987.472770


In [21]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
242245,242245,242245,2022-05-20,no,Andorra,42572.0,inf,42655.2,3.407885e+02
242246,242246,242246,2022-05-20,no,Algeria,265847.0,4.089856e+04,265846.8,4.358044e+04
242247,242247,242247,2022-05-20,no,Albania,275732.0,inf,275754.2,1.036570e+04
242248,242248,242248,2022-05-20,no,Argentina,9135308.0,inf,9135308.0,-3.269654e+15
242249,242249,242249,2022-05-20,no,Zimbabwe,250469.0,1.083235e+03,250479.0,9.874728e+02


In [22]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,level_0,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
240864,240864,240864,2022-05-16,no,Germany,25818405.0,581.775248,25824007.4,567.692789
241149,241149,241149,2022-05-17,no,Germany,25890456.0,326.129907,25877654.8,523.112686
241435,241435,241435,2022-05-18,no,Germany,25949175.0,395.901384,25939316.2,448.887269
241720,241720,241720,2022-05-19,no,Germany,25998085.0,482.135955,25994490.1,443.995157
242004,242004,242004,2022-05-20,no,Germany,26040460.0,569.554837,26049664.0,471.137442


In [23]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)

End of Groupby apply for large file and filtering. 

=================================================